In [27]:
import pandas as pd

df = pd.read_csv('/Users/wdgstl/GCPE/pefirms.csv')

In [28]:
us_firms = df[(df['country'].str.lower() == 'united states') & (df['website'].notna())]

In [39]:
record = us_firms.iloc[1]

website = record['website']

website

'joingardencity.com'

In [44]:
us_firms.iloc[4]

country                             united states
founded                                      2020
id                                   sugarcapital
industry         venture capital & private equity
linkedin_url    linkedin.com/company/sugarcapital
locality                            san francisco
name                                sugar capital
region                                 california
size                                         1-10
website                              sugarcap.com
Name: 16, dtype: object

In [37]:
import os
def read_txt(folder_path, filename):
    with open(os.path.join(folder_path, filename), "r", encoding="utf-8") as f:
        content = f.read().strip()
    return content

text = read_txt('/Users/wdgstl/GCPE/scraped_pages', 'bansk group.txt')

len(text[:500])

500

In [38]:
import requests

def call_mixtral(prompt):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "mixtral",
            "prompt": prompt,
            "stream": False
        }
    )
    return response.json()["response"]

# Example use
prompt = f"""You are analyzing a private equity firm's website description:

{text[:500]}

Your task is to extract only **explicitly stated** information.

Extract and return:
1. A list of the firm's **industry focus areas** — only include industries the firm clearly states it invests in.
2. For each industry, extract any **specific investment thesis areas** — such as subsectors, business types, or value creation strategies — only if these are directly mentioned in the text.

**Do not infer or assume** investment areas based on employee biographies, past experience, or general language. Only use information that is clearly supported by the provided text.

Output format:
Industry Focus Areas and Specific Theses:
- [Industry 1]
  - Thesis Area(s): [short bullet list]
- [Industry 2]
  - Thesis Area(s): [short bullet list]
(...continue for all explicitly mentioned industries)
"""


print(call_mixtral(prompt))


 Industry Focus Areas and Specific Theses:
- Consumer Packaged Goods (CPG)
  - Thesis Area(s): No specific thesis areas were directly mentioned in the text. The focus is on being a consumer-focused private investment firm with extensive experience across the global CPG landscape.


In [17]:
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup

# Expanded list of URL substrings to skip
EXCLUDE_KEYWORDS = {
    "careers", "jobs", "team", "people",           # HR/org pages
    "privacy", "terms", "legal",                   # policies
    "contact", "support", "help", "faq",           # support pages
    "blog", "news", "press", "media",              # marketing/updates
    "events", "webinar", "culture",                # corporate events/culture
    "login", "signup", "register", "subscribe",    # account flows
    "cookie", "rss", "sitemap",                     # misc
}

def crawl_site(start_url, max_pages=1000):
    visited = set()
    to_visit = [start_url]
    domain = urlparse(start_url).netloc

    while to_visit and len(visited) < max_pages:
        url = to_visit.pop(0)
        if url in visited:
            continue
        visited.add(url)
        print(url)

        try:
            resp = requests.get(url, timeout=5)
            if "text/html" not in resp.headers.get("Content-Type", ""):
                continue
            soup = BeautifulSoup(resp.text, "html.parser")
        except Exception:
            continue

        for tag in soup.find_all("a", href=True):
            href = urljoin(start_url, tag["href"].split('#')[0])
            parsed = urlparse(href)
            normalized = parsed._replace(query="", fragment="").geturl()
            path_lower = normalized.lower()

            # same‑domain only
            if parsed.netloc != domain:
                continue

            # skip any URL containing an exclusion keyword
            if any(kw in path_lower for kw in EXCLUDE_KEYWORDS):
                continue

            if normalized not in visited and normalized not in to_visit:
                to_visit.append(normalized)

if __name__ == "__main__":
    crawl_site("https://example-pe-firm.com", max_pages=500)


https://coreandmain.com
https://coreandmain.com/
https://coreandmain.com/who-we-are/vision-mission-values/
https://coreandmain.com/who-we-are/leadership/
https://coreandmain.com/who-we-are/mergers-acquisitions/
https://coreandmain.com/who-we-are/esg/
https://coreandmain.com/what-we-do/pipe-valves-fittings/
https://coreandmain.com/what-we-do/fire-protection/
https://coreandmain.com/what-we-do/fusible-piping/
https://coreandmain.com/what-we-do/metering-amr-ami/
https://coreandmain.com/what-we-do/storm-drain/
https://coreandmain.com/what-we-do/geosynthetics/
https://coreandmain.com/what-we-do/treatment-plants/
https://coreandmain.com/what-we-do/supply/
https://coreandmain.com/why-choose-us/case-studies/
https://coreandmain.com/why-choose-us/industry-links/
https://coreandmain.com/why-choose-us/vendor-partners/
https://coreandmain.com/why-choose-us/online-advantage/
https://coreandmain.com/overview
https://coreandmain.com/news
https://coreandmain.com/events
https://coreandmain.com/contact-

KeyboardInterrupt: 

In [104]:
import psycopg2

conn = psycopg2.connect(
    host="127.0.0.1",         
    port="5432",              
    database="firms",   
    user="postgres",          
    password="GuilphMonlrc@22Wdgcwg22"  
)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS firms (
    id SERIAL PRIMARY KEY,
    name TEXT NOT NULL,
    website TEXT,
    thesis TEXT
);
""")

conn.commit()


In [109]:
def save_firm_to_db(name: str, website: str, thesis: str):
    try:
        conn = psycopg2.connect(
            host="localhost",
            port="5432",
            database="firms",
            user="postgres",
            password="GuilphMonlrc@22Wdgcwg22"
        )
        cur = conn.cursor()
        
        cur.execute("""
            INSERT INTO firms (name, website, thesis)
            VALUES (%s, %s, %s)
        """, (name, website, thesis))

        conn.commit()
        print(f"Saved: {name}")
        
    except Exception as e:
        print("Error saving firm:", e)
    finally:
        cur.close()
        conn.close()


In [111]:
query = "SELECT * FROM firms;"

cursor.execute(query)

rows = cursor.fetchall()

for row in rows:
    print(row)

cursor.close()
conn.close()

(1, 'bansk group', 'banskgroup.com', 'test')
(2, 'garden city companies', 'joingardencity.com', 'test')
(3, 'sileo capital', 'sileocapital.com', 'test')
(4, 'intonation ventures', 'intonationventures.com', 'test')
(5, 'sugar capital', 'sugarcap.com', 'test')
(6, 'fuul capital', 'fuulcapital.com', 'test')


In [110]:
#want to do this in paralell
for i in range(len(us_firms)):
    name = us_firms.iloc[i]['name']
    website = us_firms.iloc[i]['website']
    save_firm_to_db(name, website, 'test')
    # crawl_website(name, f'https://www.{website}')
    if i == 5:
        break
    

Saved: bansk group
Saved: garden city companies
Saved: sileo capital
Saved: intonation ventures
Saved: sugar capital
Saved: fuul capital


In [90]:
cursor.execute("SELECT * FROM firms")
rows = cursor.fetchall()

for row in rows:
    print(row)

(1, 'bansk group', 'banskgroup.com', 'test')
(2, 'garden city companies', 'joingardencity.com', 'test')
(3, 'sileo capital', 'sileocapital.com', 'test')
(4, 'intonation ventures', 'intonationventures.com', 'test')
(5, 'sugar capital', 'sugarcap.com', 'test')
(6, 'fuul capital', 'fuulcapital.com', 'test')
